In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
2,application_1613331100983_0003,pyspark,idle,Link,Link


SparkSession available as 'spark'.

In [2]:
import hashlib
from datetime import datetime
from pyspark.sql import functions as F
from pyspark.sql.types import FloatType
import hsfs

In [3]:
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

## Load transactions feature group from hsfs

In [4]:
transactions_fg = fs.get_feature_group("transactions_fg", 1)
transactions_df = transactions_fg.read()
transactions_df.show()

An error was encountered:
An error occurred while calling o146.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 4 times, most recent failure: Lost task 0.3 in stage 0.0 (TID 3, amlsim-worker-1.internal.cloudapp.net, executor 1): org.apache.hadoop.hdfs.BlockMissingException: Could not obtain block: BP-1898712143-10.0.0.6-1613062180629:blk_30057_1001 file=/apps/hive/warehouse/amlsim_featurestore.db/transactions_fg_1/part-00000-23505a1a-2d50-4558-888f-758de98d6161-c000
	at org.apache.hadoop.hdfs.DFSInputStream.chooseDataNode(DFSInputStream.java:955)
	at org.apache.hadoop.hdfs.DFSInputStream.blockSeekTo(DFSInputStream.java:608)
	at org.apache.hadoop.hdfs.DFSInputStream.readWithStrategy(DFSInputStream.java:856)
	at org.apache.hadoop.hdfs.DFSInputStream.read(DFSInputStream.java:907)
	at org.apache.hadoop.hdfs.DFSInputStream.read(DFSInputStream.java:709)
	at java.io.FilterInputStream.read(FilterInputStream.java:83)
	at org.apache.parq

## Load alert transactions feature group from hsfs

In [5]:
alert_transactions_fg = fs.get_feature_group("alert_transactions_fg", 1)
alert_transactions_df = alert_transactions_fg.read()
alert_transactions_df.show()

+--------------+------+-------+--------+
|    alert_type|is_sar|tran_id|alert_id|
+--------------+------+-------+--------+
|gather_scatter|  true|  11873|      47|
|gather_scatter|  true|  11874|      47|
|gather_scatter|  true|  11875|      47|
|gather_scatter|  true|  13151|      47|
|gather_scatter|  true|  23148|      47|
|scatter_gather|  true|  23779|      17|
|scatter_gather|  true|  23780|      17|
|scatter_gather|  true|  26441|      17|
|scatter_gather|  true|  26442|      17|
|gather_scatter|  true|  28329|      47|
|gather_scatter|  true|  31581|      47|
|gather_scatter|  true|  34310|      47|
|scatter_gather|  true|  34433|      17|
|gather_scatter|  true|  36131|      58|
|scatter_gather|  true|  36563|      17|
|scatter_gather|  true|  41430|      17|
|scatter_gather|  true|  42363|      17|
|gather_scatter|  true|  42511|      58|
|gather_scatter|  true|  44370|      58|
|gather_scatter|  true|  46176|      58|
+--------------+------+-------+--------+
only showing top

## Load party feature group from hsfs

In [6]:
party_fg = fs.get_feature_group("party_fg", 1)
party_df = party_fg.read()
party_df.show()

KeyboardInterrupt: 

## Create graph edge training dataset

In [ ]:
alert_transactions_df.count()

In [ ]:
transactions_df.count()

In [ ]:
edges = transactions_df.join(alert_transactions_df,["tran_id"],"left")
edges = edges.withColumn("is_sar",F.when(F.col("is_sar") == "true", 1).otherwise(0))\
             .select("src","dst","tx_type","base_amt","tran_id","is_sar")\
             .toDF('source', 'target', 'tx_type', 'base_amt', 'tran_id', 'is_sar')

In [ ]:
edges.show()

In [ ]:
edges.count()

In [ ]:
edges.where(F.col("is_sar")==1).count()

In [ ]:
edges_td_meta = fs.create_training_dataset(name="edges_td",
                                       version=1,
                                       data_format="csv",
                                       label = ["is_sar"],   
                                       description="edges training dataset")
edges_td_meta.save(edges)

## Create graph node training dataset

In [ ]:
sources = edges.select(["source"]).toDF("id")
targets = edges.select(["target"]).toDF("id")
nodes = sources.union(targets).dropDuplicates(subset=["id"])
nodes.show()

In [ ]:
nodes.count()

In [ ]:
nodes_td = nodes.join(party_df, ["id"])
nodes_td.count()

In [ ]:
nodes_td.show()

In [ ]:
node_td_meta = fs.create_training_dataset(name="node_td",
                                       version=1,
                                       data_format="csv",   
                                       description="node training dataset")
node_td_meta.save(nodes_td)